# Initialize

To install prerequisites, open a terminal in Jupyter and run the following commands:
```
git clone https://github.com/claudiofahey/p3_test_driver
cd p3_test_driver
pip install -e p3_data
pip install openpyxl
```
Then restart the Python kernel.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import seaborn as sns
import json
from io import StringIO
import importlib
import re

In [ ]:
import p3_data
importlib.reload(p3_data)
from p3_data import (glob_file_list , load_json_from_file, merge_dicts, plot_groups, 
    get_varying_column_names, filter_dataframe, take_varying_columns,
    load_json_records_as_dataframe, flatten_multiindex_columns)

# Load and Clean Results

In [ ]:
# Load result files from P3 Test Driver
src_files = []
src_files += ['../data/p3_test_driver/results/*.json.bz2']
raw_df = load_json_records_as_dataframe(src=src_files, ignore_error=True)

In [ ]:
def parse_images_per_sec(output):
    """Search for last occurance of: total images/sec: 8169.88"""
    try:
        for m in re.finditer('^total images/sec: ([.0-9]+)$', output, flags=re.MULTILINE):
            pass
        return float(m.groups()[0])
    except:
        return np.nan

In [ ]:
# Clean raw results
def clean_result(result):
    try:
        r = result.copy()
        r['utc_begin'] = pd.to_datetime(r['utc_begin'], utc=True)
        r['utc_end'] = pd.to_datetime(r['utc_end'], utc=True)
        r['images_per_sec'] = parse_images_per_sec(r['output'])
        r['images_per_sec_per_gpu'] = r['images_per_sec'] / r['np']
        r['storage_type'] = 'isilon'
        return pd.Series(r)
    except Exception as e:
        print('ERROR: %s: %s' % (r['test_uuid'], e))
        # raise e

In [ ]:
#r = clean_result(raw_df.iloc[-1])
#pd.DataFrame(r)

In [ ]:
clean_df = raw_df.apply(clean_result, axis=1)
clean_df = clean_df.set_index('test_uuid', drop=False)
clean_df = clean_df[clean_df.error==False]
clean_df = clean_df.sort_values(['utc_begin'])
clean_df['num_copies'] = clean_df['num_copies'].fillna(1.0)

In [ ]:
raw_other_results_df = pd.read_csv('other_benchmark_results.csv')

In [ ]:
# Clean other raw results
def clean_other_result(result):
    try:
        r = result.copy()        
        r['images_per_sec_per_gpu'] = r['images_per_sec'] / r['np']
        r['NVIDIA_TENSORFLOW_VERSION'] = str(r['NVIDIA_TENSORFLOW_VERSION'])
        return pd.Series(r)
    except Exception as e:
        #print('ERROR: %s: %s' % (r['test_uuid'], e))
        raise e

In [ ]:
clean_other_results_df = raw_other_results_df.apply(clean_other_result, axis=1)

In [ ]:
combined_df = pd.concat([clean_df, clean_other_results_df], sort=False)

# Explore data

In [ ]:
# Show list of columns
list(clean_df.columns.values)

In [ ]:
# Define columns that identify test parameters
param_cols = [
 'NVIDIA_TENSORFLOW_VERSION',
 'batch_group_size',
 'batch_size',
 'cached',
 'datasets_num_private_threads',
 'datasets_prefetch_buffer_size',
 'fp16',
 'model',
 'np',
 'num_batches',
 'num_inter_threads',
 'num_intra_threads',
 'tensorflow_benchmark_git_hash',
 'test']

In [ ]:
# Define columns that are the output of the experiments
output_cols = [
    'utc_begin',    
    'images_per_sec',
    'images_per_sec_per_gpu',
]

In [ ]:
cols = param_cols + output_cols

In [ ]:
# View most recent results
clean_df[cols].tail(3).T

In [ ]:
# Export to CSV
clean_df[cols].to_csv('results.csv')

In [ ]:
clean_df.groupby(['model']).size()

In [ ]:
# First level of filtering
filt_df = filter_dataframe(
    clean_df,
#     cached=False,
)
len(filt_df)

In [ ]:
filt1_df = filter_dataframe(
    filt_df,
    storage_type='isilon',
#     NVIDIA_TENSORFLOW_VERSION='19.09',
#     model='resnet50',
#     np=48,
    num_batches=(500,50000),
#     cached=False,
#     num_copies=150,
)
take_varying_columns(filt1_df[cols]).T

In [ ]:
filt2_df = filter_dataframe(
    combined_df,
    storage_type='na',
#     model='resnet50',
#     np=48,
#     num_batches=(500,50000),
#     cached=False,
#     num_copies=150,
)
len(filt2_df)

In [ ]:
filt3_df = pd.concat([filt1_df, filt2_df])
len(filt3_df)

In [ ]:
df = filt3_df.fillna(0).groupby([
    'NVIDIA_TENSORFLOW_VERSION',
    'fp16',
    'model',
    'np',
    'storage_type',
    'cached',
    'datasets_num_private_threads',
]).mean()[['images_per_sec_per_gpu']]
df

In [ ]:
df = df.unstack([
    'storage_type',
    'cached',
    'datasets_num_private_threads',
    'NVIDIA_TENSORFLOW_VERSION',
])
df

In [ ]:
# baseline = df[('images_per_sec_per_gpu','isilon',True,2)]
baseline = df[('images_per_sec_per_gpu','na',False,0.0,'19.02')]

In [ ]:
(df.div(baseline, axis=0) - 1.0) * 100.0

# Data to publish in WP (NGC 19.09)

In [ ]:
# First level of filtering
filt_df = filter_dataframe(
    combined_df,
)
len(filt_df)

In [ ]:
# Isilon
filt_isi1_df = filter_dataframe(
    filt_df,
    storage_type='isilon',
    NVIDIA_TENSORFLOW_VERSION='19.09',
    num_batches=(500,50000),
    cached=False,
    num_copies=150,
    datasets_num_private_threads=4,
)
take_varying_columns(filt_isi1_df[cols]).T

In [ ]:
# Linux cache
filt_cached1_df = filter_dataframe(
    filt_df,
    storage_type='isilon',
    NVIDIA_TENSORFLOW_VERSION='19.09',
    num_batches=(500,50000),
    cached=True,
    datasets_num_private_threads=4,
)
take_varying_columns(filt_cached1_df[cols]).T

In [ ]:
# Competitor 1
filt_comp1_df = filter_dataframe(
    filt_df,
    storage_type='na',
)
len(filt_comp1_df)

In [ ]:
filt3_df = pd.concat([filt_isi1_df, filt_cached1_df, filt_comp1_df])
len(filt3_df)

In [ ]:
agg_cols = [
    'NVIDIA_TENSORFLOW_VERSION',
    'cached',
    'datasets_num_private_threads',
    'fp16',
    'num_copies',
    'storage_type',
]

In [ ]:
g = filt3_df.fillna(0).groupby(['model', 'np',] + agg_cols)
df = g.agg({'images_per_sec_per_gpu': ['count','mean']})

In [ ]:
df = df.stack()
df.index.names = df.index.names[0:-1] + ['agg']
df = df.unstack(agg_cols + ['agg'])
df

In [ ]:
df.to_excel('/home/jovyan/mnt/hgfs/Temp/tensorflow_benchmark_results_dgx2.xlsx')

In [ ]:
baseline = df[('images_per_sec_per_gpu','19.09',True,4,True,1.0,'isilon','mean')]

In [ ]:
(df.div(baseline, axis=0) - 1.0) * 100.0

# Comparison with other products (NGC 19.02)

In [ ]:
# First level of filtering
filt_df = filter_dataframe(
    combined_df,
)
len(filt_df)

In [ ]:
# Isilon
filt_isi1_df = filter_dataframe(
    filt_df,
    storage_type='isilon',
    NVIDIA_TENSORFLOW_VERSION='19.02',
    num_batches=(500,50000),
    cached=False,
    num_copies=150,
    datasets_num_private_threads=4,
)
take_varying_columns(filt_isi1_df[cols]).T

In [ ]:
# Linux cache
filt_cached1_df = filter_dataframe(
    filt_df,
    storage_type='isilon',
    NVIDIA_TENSORFLOW_VERSION='19.02',
    num_batches=(500,50000),
    cached=True,
    datasets_num_private_threads=4,    
)
take_varying_columns(filt_cached1_df[cols]).T

In [ ]:
# Competitor 1
filt_comp1_df = filter_dataframe(
    combined_df,
    storage_type='na',
)
len(filt_comp1_df)

In [ ]:
filt3_df = pd.concat([filt_isi1_df, filt_cached1_df, filt_comp1_df])
len(filt3_df)

In [ ]:
agg_cols = [
    'NVIDIA_TENSORFLOW_VERSION',
    'cached',
    'datasets_num_private_threads',
    'fp16',
    'num_copies',
    'storage_type',
]

In [ ]:
g = filt3_df.fillna(0).groupby(['model', 'np',] + agg_cols)
df = g.agg({'images_per_sec_per_gpu': ['count','mean']})

In [ ]:
df = df.stack()
df.index.names = df.index.names[0:-1] + ['agg']
df = df.unstack(agg_cols + ['agg'])
df

In [ ]:
# baseline = df[('images_per_sec_per_gpu','isilon',True,2)]
baseline = df[('images_per_sec_per_gpu','19.02',False,0.0,True,20.0,'na','mean')]

In [ ]:
(df.div(baseline, axis=0) - 1.0) * 100.0